# M5 Data ELT Pipeline (Google Colab Optimized)

**Memory-Efficient Design:**
- Uses Parquet as intermediate disk storage (avoids OOM errors)
- Processes data in chunks from disk to SQLite
- Deletes temp files after completion

**Steps:**
1. Install PySpark and setup
2. Load and transform data with Spark
3. Write to Parquet (disk-based, memory-safe)
4. Read Parquet chunks and write to SQLite
5. Cleanup and verify

## Step 0: Setup (Colab-specific)

In [ ]:
# Install required packages
!pip install pyspark pyarrow -q
print("Packages installed successfully!")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


import os
os.chdir('/content/drive/MyDrive/m5-forecasting-accuracy/ELT')
print(f"Working directory: {os.getcwd()}")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import pyarrow.parquet as pq
import pandas as pd
import sqlite3
import shutil
import gc
import os

# Define all paths
script_dir = os.getcwd()
PATH_SALES = os.path.join(script_dir, "Raw Data", "sales_train_validation.csv")
PATH_CAL = os.path.join(script_dir, "Raw Data", "calendar.csv")
PATH_PRICES = os.path.join(script_dir, "Raw Data", "sell_prices.csv")
DB_PATH = os.path.join(script_dir, "database.db")
PARQUET_PATH = "/content/temp_parquet"

print("Paths configured:")
print(f"  Sales: {PATH_SALES}")
print(f"  Calendar: {PATH_CAL}")
print(f"  Prices: {PATH_PRICES}")
print(f"  Database: {DB_PATH}")
print(f"  Temp Parquet: {PARQUET_PATH}")

## Step 1: Prepare Fresh Database

In [ ]:
print("Preparing fresh database...")
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)
    print("  -> Deleted existing database")

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE sales_data (
        store_id TEXT,
        item_id TEXT,
        wm_yr_wk INTEGER,
        d TEXT,
        id TEXT,
        dept_id TEXT,
        cat_id TEXT,
        state_id TEXT,
        sales INTEGER,
        wday INTEGER,
        month INTEGER,
        year INTEGER,
        event_name_1 TEXT,
        event_type_1 TEXT,
        snap_CA INTEGER,
        snap_TX INTEGER,
        snap_WI INTEGER,
        sell_price REAL,
        PRIMARY KEY (store_id, item_id, d)
    )
""")
conn.commit()
conn.close()
print("  -> Fresh database created with PRIMARY KEY (store_id, item_id, d)")

## Step 2: Initialize Spark with Optimized Settings

In [ ]:
spark = SparkSession.builder \
    .appName("M5_Data_Prep") \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.local.dir", "/content/spark_temp") \
    .getOrCreate()

print(f"Spark session initialized!")
print(f"Spark version: {spark.version}")
print(f"Driver memory: 10g")

## Step 3: Define Schemas and Load Data

In [ ]:
# Define schemas for type safety and performance
sales_schema = StructType([
    StructField("id", StringType(), True),
    StructField("item_id", StringType(), True),
    StructField("dept_id", StringType(), True),
    StructField("cat_id", StringType(), True),
    StructField("store_id", StringType(), True),
    StructField("state_id", StringType(), True)
] + [StructField(f"d_{i}", IntegerType(), True) for i in range(1, 1914)])

calendar_schema = StructType([
    StructField("date", DateType(), True),
    StructField("wm_yr_wk", ShortType(), True),
    StructField("weekday", StringType(), True),
    StructField("wday", ByteType(), True),
    StructField("month", ByteType(), True),
    StructField("year", ShortType(), True),
    StructField("d", StringType(), True),
    StructField("event_name_1", StringType(), True),
    StructField("event_type_1", StringType(), True),
    StructField("event_name_2", StringType(), True),
    StructField("event_type_2", StringType(), True),
    StructField("snap_CA", ByteType(), True),
    StructField("snap_TX", ByteType(), True),
    StructField("snap_WI", ByteType(), True),
])

prices_schema = StructType([
    StructField("store_id", StringType(), True),
    StructField("item_id", StringType(), True),
    StructField("wm_yr_wk", ShortType(), True),
    StructField("sell_price", FloatType(), True)
])

print("Schemas defined!")

In [ ]:
df_sales = spark.read.csv(PATH_SALES, header=True, schema=sales_schema)
df_cal = spark.read.csv(PATH_CAL, header=True, schema=calendar_schema)
df_prices = spark.read.csv(PATH_PRICES, header=True, schema=prices_schema)


df_cal.cache()

print(f"Sales rows: {df_sales.count():,}")
print(f"Calendar rows: {df_cal.count():,}")
print(f"Prices rows: {df_prices.count():,}")

## Step 4: Transform - Melt and Join

In [ ]:
day_cols = [c for c in df_sales.columns if 'd_' in c]
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

stack_expr = f"stack({len(day_cols)}, " + \
             ", ".join([f"'{c}', {c}" for c in day_cols]) + \
             ") as (d, sales)"

df_melted = df_sales.select(*id_cols, F.expr(stack_expr))
print(f"Melted data will have ~{30490 * len(day_cols):,} rows")

In [ ]:
df_cal_prep = df_cal.select(
    "d", "wm_yr_wk", "wday", "month", "year", 
    "event_name_1", "event_type_1", "snap_CA", "snap_TX", "snap_WI"
)


df_sales_cal = df_melted.join(F.broadcast(df_cal_prep), on="d", how="left")


df_final = df_sales_cal.join(df_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")


columns = ["store_id", "item_id", "wm_yr_wk", "d", "id", "dept_id", "cat_id", 
           "state_id", "sales", "wday", "month", "year", "event_name_1", 
           "event_type_1", "snap_CA", "snap_TX", "snap_WI", "sell_price"]

df_final = df_final.select(*columns)
df_final.printSchema()

## Step 5: Write to Parquet

In [ ]:
if os.path.exists(PARQUET_PATH):
    shutil.rmtree(PARQUET_PATH)
    print(f"Cleaned up existing temp files")

print("Writing data to Parquet (this is disk-based and memory-safe)...")
df_final.write.parquet(PARQUET_PATH, mode="overwrite")
print("Parquet write complete!")

spark.stop()
print("Spark stopped - memory freed for SQLite writing")
gc.collect()

## Step 6: Read Parquet and Write to SQLite (Chunk by Chunk)

In [ ]:
parquet_files = [os.path.join(PARQUET_PATH, f) for f in os.listdir(PARQUET_PATH) if f.endswith('.parquet')]
print(f"Found {len(parquet_files)} parquet files to process")

# Open SQLite connection
conn = sqlite3.connect(DB_PATH)
total_rows = 0

for i, pq_file in enumerate(parquet_files, 1):
    print(f"\n[{i}/{len(parquet_files)}] Processing: {os.path.basename(pq_file)}")
    
    try:
        # Read parquet file
        table = pq.read_table(pq_file)
        df = table.to_pandas()
        row_count = len(df)
        
        # Write to SQLite in chunks
        df.to_sql('sales_data', conn, if_exists='append', index=False, chunksize=50000)
        total_rows += row_count
        
        print(f"   -> Written {row_count:,} rows (Total: {total_rows:,})")
        
        # Free memory
        del df, table
        gc.collect()
        
    except Exception as e:
        print(f"   -> ERROR: {e}")

print(f"\n{'='*50}")
print(f"Total rows written: {total_rows:,}")
print(f"{'='*50}")

## Step 7: Verify Data and Create Indexes

In [ ]:
cursor = conn.cursor()

# Verify row count
cursor.execute("SELECT COUNT(*) FROM sales_data")
db_rows = cursor.fetchone()[0]
print(f"Total rows in database: {db_rows:,}")


cursor.execute("""
    SELECT store_id, item_id, d, COUNT(*) as cnt 
    FROM sales_data 
    GROUP BY store_id, item_id, d 
    HAVING cnt > 1 
    LIMIT 5
""")
duplicates = cursor.fetchall()
if duplicates:
    print(f"WARNING: Found duplicates: {duplicates}")
else:
    print("No duplicates found - data is unique!")

In [ ]:
cursor.execute("SELECT * FROM sales_data LIMIT 5")
sample = cursor.fetchall()
col_names = [desc[0] for desc in cursor.description]

print(f"Columns: {col_names}")
print(f"\nSample rows:")
for row in sample:
    print(row)

In [ ]:
print("Creating indexes...")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_store_item ON sales_data(store_id, item_id)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_d ON sales_data(d)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_year_month ON sales_data(year, month)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_state ON sales_data(state_id)")
conn.commit()
print("Indexes created!")

## Step 8: Cleanup and Finish

In [ ]:

conn.close()


print("Cleaning up temporary Parquet files...")
if os.path.exists(PARQUET_PATH):
    shutil.rmtree(PARQUET_PATH)
    print("  -> Temp files deleted")


db_size_mb = os.path.getsize(DB_PATH) / (1024 * 1024)
print(f"\nDatabase file size: {db_size_mb:.2f} MB")

print("\n" + "="*50)
print("ALL DONE! Database is ready.")
print(f"Location: {DB_PATH}")
print("="*50)

## (Optional) Download Database

In [ ]:
# Download the database file to your local machine
from google.colab import files
files.download(DB_PATH)

## (Optional) Quick Test Query

In [ ]:
conn = sqlite3.connect(DB_PATH)
test_df = pd.read_sql_query("""
    SELECT store_id, COUNT(*) as row_count, SUM(sales) as total_sales
    FROM sales_data
    GROUP BY store_id
    ORDER BY store_id
""", conn)
conn.close()

print("Sales summary by store:")
print(test_df)